# Two Layer Neural Network
# hyperparameter tuning
This new notebook is to:
1. clean up the cs231n official note
2. isolate the tuning part of the work.

In [1]:
# A bit of setup
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
import copy

from cs231n.classifiers.neural_net import TwoLayerNet
from cs231n.classifiers.neural_net_tunning import hyper_params_comb, net_tuning

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# some of the functions that might come in handy later
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
# load CIFAR-10 data
from cs231n.data_utils import load_CIFAR10, get_CIFAR10_data

data_set = get_CIFAR10_data()

# unpack the data
X_train = data_set['X_train']
y_train = data_set['y_train']
X_val = data_set['X_val']
y_val = data_set['y_val']
X_test = data_set['X_test']
y_test = data_set['y_test']

# check dimensions
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 3072)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3072)
Test labels shape:  (1000,)


In [ ]:
# build the developing dataset
num_dev = 500
mask_dev = np.random.choice(X_train.shape[0], num_dev, replace=False)
X_dev = X_train[mask_dev]
y_dev = y_train[mask_dev]
print('Dev data shape: ', X_dev.shape)
print('Dev labels shape: ', y_dev.shape)

# use developing data if dev_mode is on
dev_mode = False  # developing mode flag: True(on) or False(off)
if dev_mode:
    X_train = X_dev
    y_train = y_dev
    print('Notice: using dev dataset!')
else:
    print('Caution: using full training dataset!')

Dev data shape:  (500, 3072)
Dev labels shape:  (500,)
Caution: using full training dataset!


**Note that all list/array of hyperparameters are arranged in the order of:  
[hidden_size, learning_rate, num_epochs, reg]**

In [ ]:
# 1.1 hidden size: coarse search

# set range of tuning for hyperparameters
hidden_size_range = np.arange(50, 151, 10)
learning_rate_range = [1e-3]
num_epochs_range = [5]
reg_range = [0.25]
# drop = 1  # single on/off flag

hyper_params_range = [hidden_size_range, learning_rate_range, num_epochs_range, reg_range]

# net model tuning
hyper_params_list = hyper_params_comb(hyper_params_range)
best_net, results = net_tuning(X_train, y_train, 
                               X_val, y_val, 
                               hyper_params_list, verbose=True)

train_acc_history = [results[hyper_params][0] for hyper_params in results]
val_acc_history = [results[hyper_params][1] for hyper_params in results]

# visualise of results: hidden_size vs train/val accuracy
plt.plot(hidden_size_range, train_acc_history, label='train')
plt.plot(hidden_size_range, val_acc_history, label='val')
plt.title('Coarse tuning of hidden size')
plt.legend()
plt.show()

iteration 100 / 1225: loss 1.956312
iteration 200 / 1225: loss 1.840139
number of epochs completed: 1
iteration 300 / 1225: loss 1.747287
iteration 400 / 1225: loss 1.628935
number of epochs completed: 2
iteration 500 / 1225: loss 1.608940
iteration 600 / 1225: loss 1.513103
iteration 700 / 1225: loss 1.585361
number of epochs completed: 3
iteration 800 / 1225: loss 1.554764
iteration 900 / 1225: loss 1.512797
number of epochs completed: 4
iteration 1000 / 1225: loss 1.580016
iteration 1100 / 1225: loss 1.679246
iteration 1200 / 1225: loss 1.448100
number of epochs completed: 5
Hyperparameter combinations completed: 1 / 11

iteration 100 / 1225: loss 1.921875
iteration 200 / 1225: loss 1.800733
number of epochs completed: 1
iteration 300 / 1225: loss 1.661400
iteration 400 / 1225: loss 1.656399
number of epochs completed: 2
iteration 500 / 1225: loss 1.628341
iteration 600 / 1225: loss 1.711735
iteration 700 / 1225: loss 1.560009
number of epochs completed: 3
iteration 800 / 1225: loss

In [ ]:
# # 1.2 hidden size: fine tuning
# # set the fine tuning range for hidden size
# # this range should be based on the result acquired in the coarse search above
# hidden_size_range_fine = [100]
# learning_rate_range = [1e-3]
# num_epochs_range = [5]
# reg_range = [0.25]

# hyper_params_range = [hidden_size_range_fine, learning_rate_range, num_epochs_range, reg_range]

# # net model tuning
# hyper_params_list = hyper_params_comb(hyper_params_range)
# best_net, results = net_tuning(X_train, y_train, X_val, y_val, hyper_params_list)


In [ ]:
# 2.1 learning rate: coarse search

In [ ]:
# 2.2 learning rate: fine tuning

In [ ]:
# 3.1 reg strength: coarse search

In [ ]:
# 3.2 reg strength: fine tuning

In [ ]:
# combined fine tuning

In [ ]:
# dropout

In [ ]:
# Adam